In [17]:
import torch
from test_utils import normalize_vector

In [18]:
# 创建一个随机的 2D 张量
v = torch.arange(12).reshape((3, 4)).cuda()
print("Original tensor:")
print(v)

Original tensor:
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]], device='cuda:0')


In [19]:
# 使用 normalize_vector 函数对张量进行归一化
v_normalized, v_mag = normalize_vector(v, return_mag=True)
print("\nNormalized tensor:")
print(v_normalized)


Normalized tensor:
tensor([[0.0000, 0.2673, 0.5345, 0.8018],
        [0.3563, 0.4454, 0.5345, 0.6236],
        [0.4182, 0.4704, 0.5227, 0.5750]], device='cuda:0')


In [22]:
v_normalized.pow(2).sum(1)

tensor([1.0000, 1.0000, 1.0000], device='cuda:0')

In [20]:
# 打印原始张量的长度
print("\nMagnitude of original tensor:")
print(v_mag)


Magnitude of original tensor:
tensor([ 3.7417, 11.2250, 19.1311], device='cuda:0')
